In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import time
import gym
import numpy
import onnx
import onnxruntime
from gym import wrappers
import os
from typing import Any, Dict, List, Optional
import glob

import argparse
import os
import time
#import caffe2.python.onnx.backend as onnx_caffe2

# import geffnet
# from base.rl.ppo import PPO

pthfile = 'pth/ckpt.20.pth'
onnxpath = 'onnx/ckpt.20.onnx'
print(torch.__version__)
print(onnx.__version__)
print(onnxruntime.__version__)

class Flatten(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, x):
        return x.view(x.shape[0], -1)

class Global_Actor(nn.Module):
    def __init__(self):
        super().__init__()
        self.actor = nn.Sequential(  # (8, G, G)
            nn.Conv2d(8, 8, 3, padding=1),  # (8, G, G)
            nn.BatchNorm2d(8),
            nn.ReLU(),
            nn.Conv2d(8, 4, 3, padding=1),  # (4, G, G)
            nn.BatchNorm2d(4),
            nn.ReLU(),
            nn.Conv2d(4, 4, 5, padding=2),  # (4, G, G)
            nn.BatchNorm2d(4),
            nn.ReLU(),
            nn.Conv2d(4, 2, 5, padding=2),  # (2, G, G)
            nn.BatchNorm2d(2),
            nn.ReLU(),
            nn.Conv2d(2, 1, 5, padding=2),  # (1, G, G)
            Flatten(),  # (G*G, )
            nn.Sigmoid(),  # added for non-negative
        )
        
    def forward(self, inputs):
        x2 = self.actor(inputs)
        return x2
    
def test():
    M = 240
    batch_size = 1
    input_shape = (8, M, M)
    
    # Initialize model with the pretrained weights
    # map_location = 'gpu' if torch.cuda.is_available() else 'cpu'
    # loaded_model = torch.load(pthfile, map_location="cpu")
    # model.actor.load_state_dict(loaded_model, strict=False)
    
    model = Global_Actor()
    model.actor.load_state_dict(torch.load(pthfile, map_location='cpu'),strict=False)
    # set the model to inference mode
    model.eval()
    
    inputs = torch.randn(1, 8, 240, 240)
    map3 = torch.zeros(1, 240, 240)
    map3[0][20][20] = 1.
    inputs[0][3] = map3
    inputs[0][7] = map3
    x = inputs
    # data type nchw
    # x = torch.rand(batch_size, *input_shape)
    input_names = ["input0"]
    output_names = ["output0"]
    optional_args = dict(keep_initializers_as_inputs=True)  # pytorch 1.3 needs this for export to succeed

    try:
        torch_out = torch.onnx._export(
            model, x, onnxpath, export_params=True, verbose=False,
            input_names=input_names, output_names=output_names, **optional_args,
            opset_version=10)
    except TypeError:
        # fallback to no keep_initializers arg for pytorch < 1.3
        torch_out = torch.onnx._export(
            model, x, onnxpath, export_params=True, verbose=False,
            input_names=input_names, output_names=output_names, opset_version=10)
    '''
    # # Export the model
    torch.onnx.export(model, x, 
                      onnxpath, 
                      opset_version=10)
                      #keep_initializers_as_inputs=True, 
                      # verbose=True
     '''
    print("=============Successful==========")

1.2.0
1.6.0
1.6.0


In [2]:
test()

=============Successful==========
